<H2>課題4: 評判分析</H2>

<p>本課題ではAmazonに投稿された映画のレビュー(英語)を分析し、レビューがPositiveかNegativeかの判別を行います。</p>
<p>Amazon_reviewファイルにはTraining dataとTest dataが入っています。</p>
<p>Training dataを用いて機械学習を行い、その結果を元に6つのTest dataがpositiveかNegativeかを判別してください。</p>
<p>6章で学んだ内容を踏まえ、各セルに'#コメント'の内容を実行するコードを記入してください。</p>

<H2>1. 必要なモジュールの読み込み</H2>

In [1]:
import collections
import numpy as np
import numpy as np_amazon

from sklearn.feature_extraction import DictVectorizer

from sklearn import svm, naive_bayes
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

import string
from string import punctuation

import glob

<H2>2. ディレクトリの確認</H2>

In [2]:
# カレントディレクトリの確認
import os
print( os.listdir(os.path.normpath("./")) )

['Assignment4_asnswered.ipynb', '.ipynb_checkpoints', '_DS_Store', 'Amazon_review', 'Assignment4.ipynb']


In [3]:
# データディレクトリの確認
print( os.listdir(os.path.normpath("./Amazon_review/")) )

['Training_data', '_DS_Store', 'Test_data']


<H2>3. Dataの読み込み </H2>

フォルダ中のテキストへのパスを取得した上で各ファイルを読み込みます。

In [4]:
# default locale を ja_JP.UTF-8に設定する
def set_locale():
    default = os.environ.get('LC_ALL')
    print( "Your default locale is", default )
    if default is None:
        os.environ.setdefault('LC_ALL', 'ja_JP.UTF-8')
        print( "Your locale is set as ja_JP.UTF-8" )

set_locale()

Your default locale is None
Your locale is set as ja_JP.UTF-8


In [5]:
# globでディレクトリを検索し、フォルダ中のテキストのパスを取得 (Training set)

DATA_PATH = './Amazon_review/Training_data/'

Training_neg_files = glob.glob( os.path.normpath("{0}neg/*".format(DATA_PATH)) )
Training_pos_files = glob.glob( os.path.normpath("{0}pos/*".format(DATA_PATH)) )

In [6]:
# 相対パスで各テキストデータへのパスが格納されていることを確認
print(Training_neg_files[0:2])
print(Training_pos_files[0:2])

['Amazon_review/Training_data/neg/cv266_tok-5311.txt', 'Amazon_review/Training_data/neg/cv032_tok-9567.txt']
['Amazon_review/Training_data/pos/cv048_tok-12726.txt', 'Amazon_review/Training_data/pos/cv018_tok-10094.txt']


In [7]:
# globでディレクトリを検索し、フォルダ中のテキストのパスを取得 (Test set)

DATA_PATH = './Amazon_review/Test_data/'

Test_neg_files = glob.glob( os.path.normpath("{0}neg/*".format(DATA_PATH)) )
Test_pos_files = glob.glob( os.path.normpath("{0}pos/*".format(DATA_PATH)) )

In [8]:
# 相対パスで各テキストデータへのパスが格納されていることを確認
print(Test_neg_files[0:2])
print(Test_pos_files[0:2])

['Amazon_review/Test_data/neg/amazon_review_10000.txt', 'Amazon_review/Test_data/neg/amazon_review_10002.txt']
['Amazon_review/Test_data/pos/amaozn_review_20001.txt', 'Amazon_review/Test_data/pos/amaozn_review_20000.txt']


In [9]:
#  各ファイルを読み込む関数を定義
import sys

def text_reader(file_path):
    python_version = sys.version_info.major
    
    if python_version >= 3:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                print(line)
    else:
        with open(file_path, 'r') as f:
            for line in f:
                print(line)

In [10]:
# 読み込んだファイルの中身を表示（Test data, Negative, 1ファイル目）
text_reader(Test_neg_files[0])

The Bad: Disregard for plot points from the lead up movies, including the first Avengers (mentioned, but stripped of all layers). Characters having sudden new and unexplained personalities (bound to happen when they're juggling so damned many!). CGI is a wonderful tool, but when you see Spider-Man's head floating above his CGI body (no he did not get decapitated), it pulls you out of the narrative. The focus of the film was on too many characters who were not relevant to the central plot, maybe the excuse was to have an epic Final Battle scene in Wakanda (SPOILER: Some of those dead characters are needed for a Black Panther sequel, further proving my point).



The Ugly: One character death after another, with so many dying that they often skip to the aftermath or just state that Thanos killed them, it really pulls at the heartstrings... Except, as this review title implies, the movie has zero lasting effects. They wasted no time in throwing out all credibility (SPOILER: Thor caused th

<H2> 4.特徴ベクトルの作成</H2>
<p>特徴ベクトルの作成に必要な関数を定義した後に特徴ベクトルを作成します。</p>
<p>特徴ベクトルはTraining setとTest setで同じ要素を持つ必要がありますので、両方のsetを合わせて作成し、後でTraining setとTest setに分けます。</p>

In [11]:
# 各単語の出現回数をカウントする関数を定義
def word_counter(string):
    words = string.strip().split()
    count_dict = collections.Counter(words)
    return dict(count_dict)

In [12]:
# unigramを作成する関数を定義
def get_unigram(file_path):
    result = []
    python_version = sys.version_info.major
    
    if python_version >= 3:
        for file in file_path:
            with open(file, 'r', encoding='utf-8') as f:
                text = f.read()
                exclude = list(string.punctuation)+["“"]+["\n"]
                text = ''.join(ch for ch in text if ch not in exclude)
                
                count_dict = word_counter(text)
                result.append(count_dict)
                
    else:
        for file in file_path:
            with open(file, 'r') as f:
                for line in f:
                    count_dict = word_counter(line)
                    result.append(count_dict)
    
    return result

In [13]:
# Unigramをリストアップ (Training set + Test set)

#%%time
Training_NUM = 700
Test_NUM = 3
unigrams_data = get_unigram(Training_neg_files[:Training_NUM]) + get_unigram(Training_pos_files[:Training_NUM]
                                                                                     + Test_neg_files[:Test_NUM]) + get_unigram(Test_pos_files[:Test_NUM])


In [14]:
# Unigramの内容を確認 (Training set)
print(unigrams_data[0] )
print("data size :", sys.getsizeof(unigrams_data) / 1000000, "[MB]" )

{'actually': 1, 'im': 1, 'fairly': 1, 'sure': 3, 'the': 38, 'experience': 2, 'of': 20, 'having': 3, 'my': 4, 'flesh': 1, 'torn': 1, 'and': 17, 'mutilated': 1, 'by': 2, 'barbed': 1, 'wire': 7, 'would': 2, 'have': 4, 'been': 1, 'more': 5, 'positive': 1, 'than': 5, 'watching': 1, 'this': 5, 'movie': 8, 'baywatch': 1, 'babe': 3, 'pamela': 8, 'anderson': 3, 'lee': 1, 'proves': 1, 'once': 2, 'for': 6, 'all': 3, 'that': 7, 'she': 8, 'should': 1, 'keep': 1, 'her': 15, 'doubleds': 1, 'on': 4, 'small': 1, 'screen': 2, 'at': 3, 'least': 1, 'there': 4, 'you': 5, 'dont': 3, 'to': 14, 'pay': 3, 'see': 2, 'cleavage': 2, 'those': 3, 'viewers': 1, 'out': 4, 'who': 4, 'only': 5, 'lay': 1, 'down': 2, 'money': 2, 'in': 14, 'hopes': 1, 'seeing': 2, 'pam': 3, 'topless': 2, 'hate': 1, 'burst': 1, 'your': 1, 'bubble': 1, 'but': 7, 'are': 2, 'no': 2, 'fullfledged': 1, 'nude': 3, 'scenes': 4, 'barb': 6, 'wouldnt': 2, 'be': 3, 'reading': 1, 'review': 1, 'right': 2, 'now': 1, 'if': 6, 'i': 9, 'had': 1, 'known': 1

In [15]:
# DictVectorizerを用いてUnigramを行列の形に変形　（各行が1つのレビュー、各列が単語、要素がその単語の出現数）

#%%time
vec = DictVectorizer()

feature_vectors_csr = vec.fit_transform(unigrams_data)

In [16]:
# 作成したデータを確認
feature_vectors_csr

<1406x41984 sparse matrix of type '<class 'numpy.float64'>'
	with 483402 stored elements in Compressed Sparse Row format>

## 5.ラベルデータの作成

今回扱うデータセットは全てに negative, positive というラベルが振られています。<br>
ここではそのラベルを negative → 0, positive → 1 とすることで二値判別問題のセットアップを構築します。<br>
ラベルも特徴ベクトルと同様にTraining setとTest setを合わせて作成し、後でTraining setとTest　setに分けます。

In [17]:
# ラベルデータの作成
labels = np.r_[np.tile(0, Training_NUM), np.tile(1, Training_NUM), np.tile(0, Test_NUM), np.tile(1, Test_NUM)]

In [18]:
#正しい位置で0と1の振替がなされているか確認 (Training setの0と1、Test setの0と1)
print( labels[0], labels[Training_NUM-1], labels[Training_NUM], labels[2*Training_NUM-1],
      labels[2*Training_NUM], labels[2*Training_NUM+Test_NUM-1], labels[2*Training_NUM+Test_NUM], labels[2*Training_NUM+2*Test_NUM-1])

0 0 1 1 0 0 1 1


<H2> 6. 学習用データと学習評価用データの作成</H2>
<p>今回はTree fold cross validationを行いますので、まずTraining setを3分割します。</p>

In [19]:
# 今回はThree fold cross validation でモデルを評価
N_FOLDS = 3

In [20]:
# seedを固定し、乱数を1400個作成
np.random.seed(7789)

shuffle_order = np.random.choice( 2*Training_NUM, 2*Training_NUM, replace=False )

In [21]:
#生成した乱数の中身を確認
print( "length :", len(shuffle_order) )
print( "first 10 elements :", shuffle_order[0:10] )

length : 1400
first 10 elements : [1235 1232  910  162  343 1160  221  545 1112 1322]


In [22]:
#作成した乱数を元にTraining setを3分割する
one_third_size = int( 2*Training_NUM / 3. )
print( "one third of the length :", one_third_size )

print( "# of '0(Positive)' in 1st set :", one_third_size - np.sum( labels[ shuffle_order[:one_third_size] ]  ) )
print( "# of '0(Positive)' in 2nd set :", one_third_size - np.sum( labels[ shuffle_order[one_third_size:2*one_third_size] ]  ) )
print( "# of '0(Positive)' in 3rd set :", one_third_size - np.sum( labels[ shuffle_order[2*one_third_size:] ]  ) )
print( "# of '1(Negative)' in 1st set :", np.sum( labels[ shuffle_order[:one_third_size] ]  ) )
print( "# of '1(Negative)' in 2nd set :", np.sum( labels[ shuffle_order[one_third_size:2*one_third_size] ]  ) )
print( "# of '1(Negative)' in 3rd set :", np.sum( labels[ shuffle_order[2*one_third_size:] ]  ) )

one third of the length : 466
# of '0(Positive)' in 1st set : 239
# of '0(Positive)' in 2nd set : 233
# of '0(Positive)' in 3rd set : 226
# of '1(Negative)' in 1st set : 227
# of '1(Negative)' in 2nd set : 233
# of '1(Negative)' in 3rd set : 240


## 7. 学習に必要な関数の設定

1. 与えられたリストをN分割する関数<br>

In [23]:
# 与えられたリストをN分割する関数を定義
def N_splitter(seq, N):
    avg = len(seq) / float(N)
    out = []
    last = 0.0
    
    while last < len(seq):
        out.append( seq[int(last):int(last + avg)] )
        last += avg
        
    return np.array(out)

In [24]:
# 動作を確認
N_splitter(range(14), 3)

array([range(0, 4), range(4, 9), range(9, 14)], dtype=object)

<p>2. train_model : 説明変数とラベルと手法を与えることでモデルを学習する</p>

In [25]:
# train_model 
def train_model(features, labels, method='SVM', parameters=None):
    ### set the model
    if method == 'SVM':
        model = svm.SVC()
    elif method == 'NB':
        model = naive_bayes.GaussianNB()
    elif method == 'RF':
        model = RandomForestClassifier()
    else:
        print("Set method as SVM (for Support vector machine), NB (for Naive Bayes) or RF (Random Forest)")
    ### set parameters if exists
    if parameters:
        model.set_params(**parameters)
    ### train the model
    model.fit( features, labels )
    ### return the trained model
    return model

<p>3. predict : モデルと説明変数を与えることでラベルを予測する</p>

In [26]:
# predict 
def predict(model, features):
    predictions = model.predict( features )
    return predictions

<p>4. evaluate_model : 予測したラベルと実際の答えの合致数を調べる

In [27]:
#evaluate_model 
def evaluate_model(predictions, labels):
    data_num = len(labels)
    correct_num = np.sum( predictions == labels )
    return data_num, correct_num

<p> 5. cross_validate : cross_validationを実行する </p>

In [28]:
# cross_validate
def cross_validate(n_folds, feature_vectors, labels, shuffle_order, method='SVM', parameters=None):
    result_test_num = []
    result_correct_num = []
    
    n_splits = N_splitter( range(2*Training_NUM), n_folds )

    for i in range(n_folds):
        print( "Executing {0}th set...".format(i+1) )
        
        test_elems = shuffle_order[ n_splits[i] ]
        train_elems = np.array([])
        train_set = n_splits[ np.arange(n_folds) !=i ]
        for j in train_set:
            train_elems = np.r_[ train_elems, shuffle_order[j] ]
        train_elems = train_elems.astype(np.integer)

        # train
        model = train_model( feature_vectors[train_elems], labels[train_elems], method, parameters )
        # predict
        predictions = predict( model, feature_vectors[test_elems] )
        # evaluate
        test_num, correct_num = evaluate_model( predictions, labels[test_elems] )
        result_test_num.append( test_num )
        result_correct_num.append( correct_num )
    
    return result_test_num, result_correct_num

## 8. 学習の実施、精度の検証

In [29]:
# Trainingを実施

#%%time

svm_model = train_model(
    features=feature_vectors_csr[shuffle_order[0:950],:]
    , labels=labels[shuffle_order[0:950]]
    , method='SVM'
    , parameters=None
)

In [30]:
# 結果を表示
print("data : " ,shuffle_order[970:980])
print("correct label : ", labels[shuffle_order[970:980]])
print("predict label : ", predict(svm_model, feature_vectors_csr[shuffle_order[970:980], :])  )

data :  [ 490 1276  794  892  504  463   48  289 1218 1137]
correct label :  [0 1 1 1 0 0 0 0 1 1]
predict label :  [0 1 0 0 0 0 0 0 1 1]


## 9. パラメータのチューニング

In [31]:
#パラメータをチューニング（どのような形でも構いません）
#%%time

search_parameters = [
    {'kernel': ['rbf'], 'gamma': [1e-2, 1e-3, 1e-4], 'C': [0.1, 1, 10, 100, 1000]},
    {'kernel': ['linear'], 'C': [0.1, 1, 10, 100, 1000]}
]

model = svm.SVC()
clf = GridSearchCV(model, search_parameters)
clf.fit(feature_vectors_csr[0:1400], labels[0:1400] )

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'kernel': ['rbf'], 'gamma': [0.01, 0.001, 0.0001], 'C': [0.1, 1, 10, 100, 1000]}, {'kernel': ['linear'], 'C': [0.1, 1, 10, 100, 1000]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [32]:
# 結果を表示
print("best paremters : ", clf.best_params_)
print("best scores : ", clf.best_score_)

best paremters :  {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
best scores :  0.7914285714285715


In [33]:
# チューニング後のパラメータで学習

#%%time
ans, corr = cross_validate(N_FOLDS, feature_vectors_csr[0:1400], labels[0:1400], shuffle_order, method='SVM', parameters=clf.best_params_)

Executing 1th set...
Executing 2th set...
Executing 3th set...


In [34]:
# 結果を表示
print( "average precision : ", np.around( 100.*sum(corr)/sum(ans), decimals=1 ), "%" )

average precision :  78.3 %


## 10. Test dataを用いてモデルを評価

In [35]:
# 上記で作成したモデルを用いてTest dataの評価を実施
# %%time

svm_model = train_model(
    features=feature_vectors_csr[1400:1406]
    , labels=labels[1400:1406]
    , method='SVM'
    , parameters=None
)

In [36]:
# 結果を表示
print("correct label : ", labels[1400:1406])
print( "predict label : ", predict(svm_model, feature_vectors_csr[1400:1406]) )

correct label :  [0 0 0 1 1 1]
predict label :  [0 0 0 1 1 1]
